In [ ]:
%pip install retriv -qqq
%pip install nltk --force
%pip install scipy --force
%pip install numpy==1.24

In [17]:
import retriv
retriv.set_base_path("./")

In [18]:
import os
import pandas as pd
files = os.listdir('/kaggle/input/wiki-20220301-en-sci')
df = pd.DataFrame()
for file in files:
    df = pd.concat([df,pd.read_parquet('/kaggle/input/wiki-20220301-en-sci/'+file)])

In [19]:
df['id'] = df.index
del df['url']

In [ ]:
get_paras = lambda text : [p for p in text.split('\n') if len(p.split())>4]
df['text'] = df.text.apply(get_paras)
df = df.explode('text')
df = df.reset_index(drop=True)
df['id'] = df.index

In [ ]:
df = df.dropna()
collection = df.to_dict(orient='records')    

In [ ]:
from retriv import SparseRetriever

# bm25 = SparseRetriever("Training_data").index(collection)
bm25 = SparseRetriever.load("Training_data")

In [ ]:
bm25.search("Which of the following is an accurate definition of dynamic scaling in self-similar systems? </s> Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots")

In [36]:
questions = pd.read_csv(r"/kaggle/input/kaggle-llm-science-exam/train.csv")

In [37]:
questions['query'] = questions['prompt'] + ". " + questions.A 

In [65]:
_df = questions['query'].to_frame()
_df['id'] = _df.index
_df = _df.rename(columns={"query":"text"})
_queries = _df.to_dict(orient='records')

In [72]:
contexts = questions['query'].apply(lambda x: bm25.search(x, cutoff=5))

In [79]:
contexts = contexts.apply(lambda x:"</s>".join( [e['text'] for e in x]))
questions['contexts'] = contexts

In [82]:
df = questions
df["master"] = df["prompt"] + "</s>" + df["A"] + "</s>" + df["B"] + "</s>"+ df["C"] + "</s>" + df["D"] + "</s>" + df["E"] + '</s>' + df["contexts"]

In [85]:
le = {'A':0,'B':1,'C':2,'D':3,'E':4}
df['answer_le'] = df.answer.replace(ohe)

In [87]:
train_questions = df.sample(150)
test_questions = df[~(df.index.isin(train_questions.index))]

In [89]:
df['split'] = 'train'
df.loc[df.sample(50).index,"split"] = "test"

In [91]:
df.to_parquet("master_train.parquet",index=False)

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, LongT5EncoderModel
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import SequenceClassifierOutput

In [ ]:
t5_encoder = LongT5EncoderModel.from_pretrained("google/longt5-local-base")
tokenizer = AutoTokenizer.from_pretrained("google/longt5-local-base")
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(t5_encoder.parameters())

In [ ]:
class Config:
    def __init__(self,num_labels:int ,problem_type:str ,hidden_size:int ):
        self.problem_type = problem_type
        self.num_labels = num_labels
        self.hidden_size = hidden_size
        

In [ ]:
class T5EncoderClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, hidden_states, **kwargs):
        hidden_states = hidden_states[:, 0, :]  # take <s> token (equiv. to [CLS])
#         hidden_states = self.dropout(hidden_states)
        print(hidden_states.shape)
        hidden_states = self.dense(hidden_states)
        hidden_states = self.out_proj(hidden_states)
        return hidden_states




In [ ]:
class T5EncoderForSequenceClassification:
    """
    Use an in-memory T5Encoder to do sequence classification"""
    def __init__(self, t5_encoder, config):
        self.num_labels = config.num_labels
        self.config = config

        self.encoder = t5_encoder  # already initialized model
        # either we are in eval mode, and the following code should do nothing
        # or we are training, but we only want to fine tune the classifier head
        # we do not want to fine-tune the encoder
        for param in self.encoder.parameters():
            param.requires_grad = False
        self.classifier = T5EncoderClassificationHead(config)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_hidden_states=None,
        output_attentions=None,
        return_dict=None,
    ):
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ..., config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
       # return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        encoder_outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = encoder_outputs[0]
        #print(sequence_output.shape)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + encoder_outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
        )

In [ ]:
config = Config(5,"multi_label_classification",768)
model = T5EncoderForSequenceClassification(t5_encoder,config)

In [ ]:
import pandas as pd
df = pd.read_csv("/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv")

In [ ]:
# Qn : A : B : C : D : E : C1: C2 : C3 : C4 : C5

In [ ]:
df["master"] = df["prompt"] + "</s>" + df["A"] + "</s>" + df["B"] + "</s>"+ df["C"] + "</s>" + df["D"] + "</s>" + df["E"] + '</s>'